In [1]:
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data, iris.target

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.ensemble import VotingClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [3]:
clf1 = LogisticRegression(random_state=1)
clf2 = RandomForestClassifier(random_state=1)
clf3 = KNeighborsClassifier()
# eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('knn', clf3)])

# eclf.get_params().keys()

In [4]:
weights = [[1,1,1], [2,1,1], [1,2,1], [1,1,2], [2,2,1], [2,1,2], [1,2,2]]

In [5]:
# params = {'lr__C': Real(1e-6, 1e6, prior='log-uniform'),
#           'rf__max_features': Integer(1,4),
#           'knn__n_neighbors': Integer(1,20),
#           'weights': Categorical(weights), 
#           'voting': ['soft', 'hard']}

Hack to construct multiple models with different weights as I can't pass in a list of lists when searching through parameter spaces using BayesSearchCV

In [6]:
from sklearn.pipeline import Pipeline

clfs = [('lr', clf1), ('rf', clf2), ('knn', clf3)]
pipe = Pipeline([('model', VotingClassifier(estimators=[('lr', clf1), 
                                                        ('rf', clf2), 
                                                        ('knn', clf3)]))])

pipe.get_params().keys()

dict_keys(['steps', 'model', 'model__estimators', 'model__n_jobs', 'model__voting', 'model__weights', 'model__lr', 'model__rf', 'model__knn', 'model__lr__C', 'model__lr__class_weight', 'model__lr__dual', 'model__lr__fit_intercept', 'model__lr__intercept_scaling', 'model__lr__max_iter', 'model__lr__multi_class', 'model__lr__n_jobs', 'model__lr__penalty', 'model__lr__random_state', 'model__lr__solver', 'model__lr__tol', 'model__lr__verbose', 'model__lr__warm_start', 'model__rf__bootstrap', 'model__rf__class_weight', 'model__rf__criterion', 'model__rf__max_depth', 'model__rf__max_features', 'model__rf__max_leaf_nodes', 'model__rf__min_impurity_split', 'model__rf__min_samples_leaf', 'model__rf__min_samples_split', 'model__rf__min_weight_fraction_leaf', 'model__rf__n_estimators', 'model__rf__n_jobs', 'model__rf__oob_score', 'model__rf__random_state', 'model__rf__verbose', 'model__rf__warm_start', 'model__knn__algorithm', 'model__knn__leaf_size', 'model__knn__metric', 'model__knn__metric_par

In [7]:
search_spaces = []

for weight in weights:
    # hacky way to test out weights is by constructing a search space
    # for each one
    params = {'model': Categorical([VotingClassifier(estimators=clfs, 
                                                     weights=weight)]),
              'model__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
              'model__rf__max_features': Integer(1,4),
              'model__knn__n_neighbors': Integer(1,20),
              'model__voting': ['soft', 'hard']}
    search_spaces.append((params, 20))

In [8]:
## was testing out individual search spaces befor using above for loop
## to construct a bunch of them
# first_srch = {'model': Categorical([VotingClassifier(estimators=clfs, 
#                                                      weights=weights[0])]),
#               'model__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
#               'model__rf__max_features': Integer(1,4),
#               'model__knn__n_neighbors': Integer(1,20),
#               'model__voting': ['soft', 'hard']}
# scnd_srch =  {'model': Categorical([VotingClassifier(estimators=clfs, 
#                                                      weights=weights[1])]),
#               'model__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
#               'model__rf__max_features': Integer(1,4),
#               'model__knn__n_neighbors': Integer(1,20),
#               'model__voting': ['soft', 'hard']}

In [10]:
grid = BayesSearchCV(pipe, search_spaces=search_spaces, cv=5, n_jobs=-1)
grid.fit(iris.data, iris.target)

/home/savvas-mint/python_dev_pkgs/scikit-optimize/skopt/optimizer/optimizer.py:363: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/savvas-mint/python_dev_pkgs/scikit-optimize/skopt/optimizer/optimizer.py:363: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/savvas-mint/python_dev_pkgs/scikit-optimize/skopt/optimizer/optimizer.py:363: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/savvas-mint/python_dev_pkgs/scikit-optimize/skopt/optimizer/optimizer.py:363: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/home/savvas-mint/python_dev_pkgs/scikit-optimize/skopt/optimizer/optimizer.py:363: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective h

In [11]:
grid.best_estimator_

Pipeline(steps=[('model', VotingClassifier(estimators=[('lr', LogisticRegression(C=847832.95827448287, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=1,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)), ('...wski',
           metric_params=None, n_jobs=1, n_neighbors=12, p=2,
           weights='uniform'))],
         n_jobs=1, voting='hard', weights=[1, 1, 2]))])

In [12]:
import pandas as pd

In [13]:
cv_results = pd.DataFrame(grid.cv_results_)

In [14]:
pd.options.display.max_columns = None

In [15]:
cv_results.sort_values('mean_test_score', ascending=False)

mean_fit_time  mean_score_time  mean_test_score  mean_train_score  \
90        0.022590         0.002729         0.980000          0.976667   
89        0.025580         0.003106         0.980000          0.975000   
127       0.022266         0.002660         0.980000          0.993333   
0         0.024278         0.003289         0.973333          0.991667   
145       0.024957         0.002583         0.973333          0.986667   
99        0.023620         0.002889         0.973333          0.990000   
31        0.023099         0.002817         0.973333          0.981667   
93        0.022461         0.002805         0.973333          0.980000   
88        0.025773         0.003417         0.973333          0.973333   
123       0.024813         0.002337         0.973333          0.980000   
124       0.023348         0.002877         0.973333          0.990000   
129       0.021593         0.003041         0.973333          0.988333   
144       0.026189         0.003512         0.973333          0.988333   
76        0.019552         0.001959         0.973333          0.978333   
9         0.023756         0.003052         0.973333          0.993333   
8         0.024241         0.003378         0.973333          0.990000   
5         0.023063         0.002586         0.973333          0.990000   
153       0.023221         0.003047         0.973333          0.991667   
63        0.023924         0.002155         0.966667          1.000000   
95        0.021705         0.002283         0.966667          1.000000   
64        0.024867         0.003254         0.966667          0.998333   
65        0.025058         0.003061         0.966667          0.998333   
91        0.024016         0.002930         0.966667          1.000000   
67        0.023822         0.002995         0.966667          1.000000   
71        0.023445         0.002728         0.966667          0.998333   
72        0.025698         0.002698         0.966667          0.980000   
82        0.020885         0.002923         0.966667          0.968333   
68        0.023210         0.002642         0.966667          0.998333   
70        0.021887         0.002697         0.966667          1.000000   
81        0.024448         0.002466         0.966667          0.985000   
..             ...              ...              ...               ...   
42        0.022972         0.002803         0.960000          0.990000   
84        0.023855         0.002963         0.960000          0.968333   
40        0.025527         0.002947         0.960000          0.983333   
100       0.023842         0.002414         0.960000          0.990000   
110       0.021723         0.002617         0.960000          1.000000   
108       0.023470         0.002768         0.960000          1.000000   
107       0.024350         0.002931         0.960000          1.000000   
39        0.018348         0.002327         0.960000          1.000000   
105       0.023747         0.003073         0.960000          0.986667   
27        0.020958         0.002621         0.960000          0.976667   
101       0.024134         0.002451         0.960000          0.993333   
30        0.021200         0.001971         0.960000          0.988333   
34        0.022761         0.002753         0.960000          1.000000   
35        0.023271         0.003077         0.960000          0.990000   
36        0.023289         0.002902         0.960000          1.000000   
37        0.021079         0.002759         0.960000          0.990000   
94        0.023100         0.002147         0.960000          1.000000   
38        0.019483         0.002477         0.960000          0.990000   
92        0.024529         0.002256         0.960000          1.000000   
111       0.022811         0.002781         0.960000          1.000000   
61        0.022429         0.002643         0.953333          1.000000   
60        0.020482         0.001905         0.953333          1.000000   
55       